<a href="https://colab.research.google.com/github/Ian-Ibrahim/SU-Machine-learning-1222785/blob/main/LSTM_3rd_JulyML_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

print(tf.__version__)

2.12.0


In [ ]:
dataset = load_dataset("nanyy1025/covid_fake_news")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/nanyy1025___csv/nanyy1025--covid_fake_news-389e2fecc7fefa61/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tweet', 'label'],
        num_rows: 6420
    })
    validation: Dataset({
        features: ['id', 'tweet', 'label'],
        num_rows: 2140
    })
    test: Dataset({
        features: ['id', 'tweet', 'label'],
        num_rows: 2140
    })
})

In [ ]:
train_texts = dataset['train']['tweet']
train_labels = dataset['train']['label']

test_texts = dataset['test']['tweet']
test_labels = dataset['test']['label']

val_texts = dataset['validation']['tweet']
val_labels = dataset['validation']['label']


print(len(train_texts))
print(len(test_texts))
print(len(val_texts))

6420
2140
2140


In [ ]:
df = pd.DataFrame()
df['text'] = train_texts + test_texts + val_texts
df['label'] = train_labels + test_labels + val_labels
df.head

<bound method NDFrame.head of                                                     text label
0      The CDC currently reports 99031 deaths. In gen...  real
1      States reported 1121 deaths a small rise from ...  real
2      Politically Correct Woman (Almost) Uses Pandem...  fake
3      #IndiaFightsCorona: We have 1524 #COVID testin...  real
4      Populous states can generate large case counts...  real
...                                                  ...   ...
10695  Donald Trump wrongly claimed that New Zealand ...  fake
10696  Current understanding is #COVID19 spreads most...  real
10697  Nothing screams “I am sat around doing fuck al...  fake
10698  Birx says COVID-19 outbreak not under control ...  fake
10699  Another 4422 new coronavirus cases have been c...  real

[10700 rows x 2 columns]>

In [ ]:
def enc(label):
  return 1 if label =='real' else 0

df['y']=df['label'].apply(lambda x: enc(x))

In [ ]:
df

,text,label,y
0,The CDC currently reports 99031 deaths. In gen...,real,1
1,States reported 1121 deaths a small rise from ...,real,1
2,Politically Correct Woman (Almost) Uses Pandem...,fake,0
3,#IndiaFightsCorona: We have 1524 #COVID testin...,real,1
4,Populous states can generate large case counts...,real,1
...,...,...,...
10695,Donald Trump wrongly claimed that New Zealand ...,fake,0
10696,Current understanding is #COVID19 spreads most...,real,1
10697,Nothing screams “I am sat around doing fuck al...,fake,0
10698,Birx says COVID-19 outbreak not under control ...,fake,0


In [ ]:
df['text'] = df['text'].str.replace('[^\w\s]', '') # removes punctuation
df['text'] = df['text'].str.replace('\d+', '') # removes numbers
df['text'] = df['text'].str.replace('@', '') # removes leading whitespace '@'
df['text'] = df['text'].str.replace('#', '') # removes trailing '#'
df['text'] = df['text'].str.replace('\s+', ' ') # removes extra whitespace
df['text'] = df['text'].str.lower() # lowercase
df.head()

<ipython-input-11-d2b5090c0f96>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]', '') # removes punctuation
<ipython-input-11-d2b5090c0f96>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\d+', '') # removes numbers
<ipython-input-11-d2b5090c0f96>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\s+', ' ') # removes extra whitespace


,text,label,y
0,the cdc currently reports deaths in general th...,real,1
1,states reported deaths a small rise from last ...,real,1
2,politically correct woman almost uses pandemic...,fake,0
3,indiafightscorona we have covid testing labora...,real,1
4,populous states can generate large case counts...,real,1


In [ ]:
train_texts,test_texts,train_labels,test_labels=train_test_split=train_test_split(df['text'],df['y'],test_size=0.2,random_state=42)

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(train_texts)
train_seq=tokenizer.texts_to_sequences(train_texts)
test_seq=tokenizer.texts_to_sequences(test_texts)

In [ ]:
max_seq_length = max(max(map(len, train_seq)),  max(map(len, train_seq)))
print(max_seq_length)

1446


In [ ]:
train_data = pad_sequences(train_seq, maxlen = max_seq_length)
test_data = pad_sequences(test_seq, maxlen = max_seq_length)

In [ ]:
print('GPU available', tf.config.list_physical_devices('GPU'))


GPU available []


In [ ]:
model=Sequential()
model.add(Embedding(len(tokenizer.word_index)+1,100,input_length=max_seq_length ))
model.add(LSTM(128))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history=model.fit(train_data)

Epoch 1/2
  9/268 [>.............................] - ETA: 10:29 - loss: 0.6877 - accuracy: 0.5139